# Hidden Markov Model (HMM)



# Graphical Summary

![fig](./cartoons/fig.svg)

# Key Formula

key formula sentence

# Technical Details

techinical details here

# Example

In [ ]:
# Clear the environment
rm(list = ls())
